<a href="https://colab.research.google.com/github/EmmaHsueh/PL_project/blob/main/VisualExpenseManagementApplication(%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA_gradio%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ----------------------------------------------------------------------
# 1. Colab 驗證與 Google Sheets 授權 (請在 Colab 環境中執行這段)
# ----------------------------------------------------------------------
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
import gradio as gr
import io
import datetime

print("正在執行 Colab 驗證...")
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("Colab 驗證完成。")

# Google Sheets 設置
GSHEET_URL = 'https://docs.google.com/spreadsheets/d/13KwCOffzsODwrwTg8SqfwUZBkvxGZveFozel_EqBI8I/edit?usp=sharing'
SHEET_NAME = '工作表1' # 假設你的工作表名稱

try:
    # 打開試算表
    gsheets = gc.open_by_url(GSHEET_URL)
    # 取得第一個工作表
    sheet = gsheets.sheet1
except Exception as e:
    print(f"無法開啟 Google Sheet 或存取工作表：{e}")
    # 建立一個模擬的 sheet 物件以防止程式完全崩潰，但在 Gradio 中會失敗
    # 實際部署時，請確保 URL 和權限正確。


正在執行 Colab 驗證...
Colab 驗證完成。


In [5]:
# ----------------------------------------------------------------------
# 2. 核心功能函數 (整合 Google Sheets I/O)
# ----------------------------------------------------------------------

In [6]:
def load_dataframe_from_sheet():
    """從 Google Sheet 讀取所有資料並轉換為 DataFrame。"""
    try:
        # 使用 get_all_records() 讀取資料，這會自動將標題行作為鍵，並嘗試推斷類型
        # 但 '金額' 欄位可能仍需手動轉換以確保計算正確
        records = sheet.get_all_records()

        if not records:
            # 如果沒有記錄，但有標題，則只創建空的 DataFrame
            headers = sheet.row_values(1)
            if headers:
                return pd.DataFrame(columns=headers)
            # 如果連標題都沒有，則使用預期的欄位名稱
            return pd.DataFrame(columns=['日期', '類別', '品項', '金額', '付款方式'])

        df = pd.DataFrame(records)

        # 確保 '金額' 欄位是數值類型
        # 由於 gspread get_all_records 可能將數值讀為 int 或 float，這裡用 to_numeric 確保一致性
        df['金額'] = pd.to_numeric(df['金額'], errors='coerce')
        df = df.dropna(subset=['金額']) # 移除無法轉換金額的行

        return df

    except Exception as e:
        # 在 Colab 環境中，如果連線失敗，回傳空的 DataFrame
        print(f"讀取 Google Sheet 發生錯誤: {e}")
        return pd.DataFrame(columns=['日期', '類別', '品項', '金額', '付款方式'])


In [7]:
def write_data_to_sheet(df):
    """將 DataFrame 寫回 Google Sheet (覆蓋現有內容)。"""
    try:
        # 準備寫入資料: 標題行 + 資料行
        data_to_write = [df.columns.values.tolist()] + df.values.tolist()

        # 清除並更新工作表
        sheet.clear()
        sheet.update(data_to_write, value_input_option='USER_ENTERED')
        print("資料已成功寫回 Google Sheet。")
        return True
    except Exception as e:
        print(f"寫入 Google Sheet 發生錯誤: {e}")
        return False

In [8]:
def record_expense_and_update(date_str: str, category: str, item: str, amount: float, payer: str):
    """
    Gradio 介面調用函數：記錄一筆支出，更新 Google Sheet，並返回更新後的 DataFrame。
    """
    try:
        # 1. 讀取現有資料
        df_expenses = load_dataframe_from_sheet()

        # 2. 創建新資料行
        new_expense = {
            '日期': date_str,
            '類別': category,
            '品項': item,
            '金額': float(amount), # 確保金額是 float
            '付款方式': payer
        }

        # 3. 添加到 DataFrame
        new_df = pd.concat([df_expenses, pd.DataFrame([new_expense])], ignore_index=True)

        # 4. 寫回 Google Sheet
        if write_data_to_sheet(new_df):
            message = f"✅ 成功記錄：{item} ({amount} 元)，資料已同步至 Google Sheet。"
        else:
            message = f"⚠️ 記錄 {item} 失敗，請檢查 Google Sheet 權限或 URL。"

        return message, new_df # Gradio 輸出：消息 + DataFrame

    except Exception as e:
        error_message = f"❌ 記錄支出時發生錯誤: {e}"
        print(error_message)
        # 返回錯誤消息和空 DataFrame 或舊 DataFrame
        return error_message, load_dataframe_from_sheet()

In [9]:
def calculate_summary_and_share(num_people: int):
    """
    Gradio 介面調用函數：計算總支出並根據人數計算分攤金額。
    """
    try:
        df_expenses = load_dataframe_from_sheet()

        if df_expenses.empty:
            return "## 🔍 支出分析結果\n\n**沒有任何支出記錄，請先新增記錄。**"

        # 確保 '金額' 是數值類型 (load_dataframe_from_sheet 已處理)
        total_amount = df_expenses['金額'].sum()

        result_html = f"## 🔍 支出分析結果\n\n"
        result_html += f"**總支出金額：** <span style='font-size: 1.5em; color: #10B981;'>{total_amount:,.2f} 元</span>\n\n"

        if num_people is None or num_people <= 0:
            result_html += "\n**請輸入有效的分攤人數（大於 0）來計算分攤金額。**"
        else:
            share_per_person = total_amount / num_people
            result_html += f"### 💰 支出分攤速算\n\n"
            result_html += f"- **分攤人數：** **{num_people}** 人\n"
            result_html += f"- **每人應分攤金額：** <span style='font-size: 1.5em; color: #EF4444;'>{share_per_person:,.2f} 元</span>"

        # (選作) 顯示類別支出彙總
        category_summary = df_expenses.groupby('類別')['金額'].sum().sort_values(ascending=False)
        result_html += "\n\n### 📊 類別支出彙總\n\n"
        for category, amount in category_summary.items():
            result_html += f"- **{category}**: {amount:,.2f} 元\n"

        return result_html

    except Exception as e:
        return f"❌ 計算發生錯誤: {e}"

In [11]:
def clear_inputs():
    """清除輸入欄位，日期設為今天，金額設為 None。"""
    today = datetime.date.today().strftime("%Y-%m-%d")
    # 依序返回：日期, 類別, 品項, 金額, 付款方式 的重設值
    return today, "", "", None, ""

In [13]:
# ----------------------------------------------------------------------
# 3. Gradio 介面配置
# ----------------------------------------------------------------------

# 初始讀取數據用於顯示
initial_df = load_dataframe_from_sheet()

# 使用 gr.Blocks 創建一個多功能的介面
with gr.Blocks(title="Google Sheets 支出管理", theme=gr.themes.Base()) as app:
    gr.Markdown(f"# 💸 支出記錄與分攤工具")
    gr.Markdown("此工具與您的 Google Sheet 同步，請確保 Colab 驗證已完成。")

    with gr.Tab("📝 記錄支出"):
        # 記錄支出區塊
        with gr.Row():
            date_input = gr.Textbox(label="日期 (YYYY-MM-DD)", placeholder=datetime.date.today().strftime("%Y-%m-%d"),
                                    value=datetime.date.today().strftime("%Y-%m-%d"), type="text")
            category_input = gr.Textbox(label="類別", placeholder="例如：餐飲、交通、娛樂")
            item_input = gr.Textbox(label="品項", placeholder="例如：午餐、車票、電影票")
            amount_input = gr.Number(label="金額", placeholder="請輸入數字")
            payer_input = gr.Textbox(label="付款方式", placeholder="例如：現金、信用卡、人名")

        submit_btn = gr.Button("記錄並同步至 Google Sheet")
        clear_btn = gr.Button("全部清除", variant="secondary")

        # 輸出區塊
        record_message = gr.Markdown("等待記錄...")
        current_data_output = gr.Dataframe(
            label="🔄 Google Sheet 最新資料 (會自動更新)",
            value=initial_df,
            interactive=False,
            wrap=True
        )

        # 定義點擊事件
        submit_btn.click(
            fn=record_expense_and_update,
            inputs=[date_input, category_input, item_input, amount_input, payer_input],
            outputs=[record_message, current_data_output]
        )

        # 2. 定義「全部清除」按鈕事件
        clear_btn.click(
            fn=clear_inputs,
            inputs=[],
            # 輸出是輸入框自身，用於重設它們的值
            outputs=[date_input, category_input, item_input, amount_input, payer_input]
        )

    with gr.Tab("📈 支出分析與分攤"):
        # 支出分析區塊
        with gr.Row():
            num_people_input = gr.Number(label="分攤人數", value=2, minimum=1)
            analyze_btn = gr.Button("計算總額與分攤")

        analysis_output = gr.Markdown("點擊 '計算總額與分攤' 以顯示結果。")

        # 定義點擊事件
        analyze_btn.click(
            fn=calculate_summary_and_share,
            inputs=[num_people_input],
            outputs=[analysis_output]
        )

        # 增加一個按鈕用於重新整理資料（例如：在其他 Tab 記錄後）
        with gr.Row():
             refresh_btn = gr.Button("重新整理最新資料")
             # 重新整理 DataFrame
             refresh_btn.click(
                 fn=load_dataframe_from_sheet,
                 inputs=[],
                 outputs=[current_data_output]
             )


# 在 Colab 環境中啟動 Gradio App
# share=True 會生成一個公開連結，方便查看
if 'google.colab' in str(get_ipython()):
    print("\n--- 正在啟動 Gradio 應用程式 ---")
    app.launch(debug=True, share=True)
else:
    app.launch(debug=True)



--- 正在啟動 Gradio 應用程式 ---
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9ab2f11d15ad3b2033.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


資料已成功寫回 Google Sheet。
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9ab2f11d15ad3b2033.gradio.live
